In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# from MCMAC_pre_merger import MCengine_pre_merger
# from MCMAC_post_merger import MCengine_post_merger 

# change prior: angle 0-45 degree
from modules.MCMAC_pre_merger_changeprior_45degree import MCengine_pre_merger
from modules.MCMAC_post_merger_changeprior_45degree import MCengine_post_merger 

import numpy as np
import pickle

# Input A56 parameters

In [2]:
# Setting simulation parameters (define prior distributions)
N_mc = 1000  # Number of Monte Carlo samples

cluster_name ='A56'

redshift = 0.30256  # Redshift from the paper

# (mean value, standard deviation)
M1 = (4.5e14, 0.8e14)  # Mass distribution of the southern subcluster
M2 = (2.8e14, 0.7e14)  # Mass distribution of the northern subcluster
Z1 = (0.30256, 0.00058)  # Redshift distribution of the southern subcluster
Z2 = (0.30256, 0.00058)  # Redshift distribution of the northern subcluster

TSC_simulation = 0.52  # Time since collision in Gyr

# when you use pre-MCMAC code
# prefix = "../RealClusters/A56/simulation_pre_output" 

# when you use post-MCMAC code
prefix = "../RealClusters/A56_0-45degree/simulation_post_output" 

result_file = "result_A56_0-45degree.csv"

## arcsecond to kpc

In [3]:
from astropy.cosmology import Planck18 as cosmo

def arcsecondToKpc(redshift, arcsecond):
    # Convert arcseconds to kiloparsecs
    kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(redshift).value / 60  # kpc per arcsec
    distance_kpc = arcsecond * kpc_per_arcsec
    return distance_kpc

In [4]:

# distance_Mpc = arcsecondToKpc(redshift, arcsecond) * 0.001 # kpc to Mpc
distance_Mpc = 0.438
D_proj = (distance_Mpc, 0.05*distance_Mpc, 0.05*distance_Mpc)  # Projected separation distance distribution (mean 408 kpc, standard deviation 14 kpc)

## Confirm it collision or not

In [5]:
# Execute simulation (derive posterior distributions of dynamical properties such as collision velocity, time since collision, etc.)
# MCengine_pre_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Probability of system being bound = 0.99500

Probability of system being boud is over 50%, then use post merger code.

In [6]:
MCengine_post_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Completed Monte Carlo iteration 10 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 100 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 1000 of 1000.
~0 minutes remaining


(array([4.65888847e+14, 4.50846566e+14, 5.35204198e+14, 5.83657425e+14,
        6.16160078e+14, 5.13047899e+14, 3.90636786e+14, 5.87677286e+14,
        4.67429607e+14, 4.52529112e+14, 3.29575036e+14, 5.05378082e+14,
        4.11585380e+14, 4.13747436e+14, 5.22044751e+14, 5.27017299e+14,
        3.83544962e+14, 4.76219355e+14, 3.59898457e+14, 4.57632099e+14,
        5.69763380e+14, 2.60948659e+14, 1.88098170e+14, 4.20990889e+14,
        4.89003591e+14, 5.58686646e+14, 5.56744179e+14, 5.01860422e+14,
        3.73775216e+14, 3.09071441e+14, 3.93165426e+14, 5.00116461e+14,
        5.45022798e+14, 5.23199779e+14, 4.62929630e+14, 3.92678028e+14,
        4.37671959e+14, 5.06470102e+14, 4.60637534e+14, 3.87223817e+14,
        3.75252927e+14, 5.97706948e+14, 3.57615030e+14, 3.66530766e+14,
        5.12467597e+14, 3.74689390e+14, 5.82786335e+14, 3.77382895e+14,
        4.63197943e+14, 2.43231744e+14, 4.40390498e+14, 3.05191126e+14,
        4.90263082e+14, 4.49510583e+14, 4.37610642e+14, 3.439252

In [7]:
import pickle
import numpy as np
import pandas as pd

# 単位の定義
units = {
    'm1': '10^14 M☉',
    'm2': '10^14 M☉',
    'z1': '',
    'z2': '',
    'dproj': 'Mpc',
    'vradobs': 'km s^{-1}',
    'alpha': 'degree',
    'v3dobs': 'km s^{-1}',
    'd3d': 'Mpc',
    'v3dcol': 'km s^{-1}',
    'dmax': 'Mpc',
    'TSM0': 'Gyr',
    'TSM1': 'Gyr',
    'T': 'Gyr',
    'prob': ''
}

# パラメータファイルのリスト
parameters = ['_m_1.pickle', '_m_2.pickle', '_z_1.pickle', '_z_2.pickle', '_d_proj.pickle', 
              '_v_rad_obs.pickle', '_alpha.pickle', '_v_3d_obs.pickle', '_d_3d.pickle', 
              '_v_3d_col.pickle', '_d_max.pickle', '_TSM_0.pickle', '_TSM_1.pickle', 
              '_T.pickle', '_prob.pickle']

results = []

for param in parameters:
    with open(prefix + param, 'rb') as f:
        data = pickle.load(f)
        if param in ['_m_1.pickle', '_m_2.pickle']:
            # 質量をe14単位に変換
            data = np.array(data) / 1e14
        
        # Medianを計算
        median = np.median(data)
        
        # 68%および95%の信頼区間を計算
        lcl_68, ucl_68 = np.percentile(data, [16, 84])
        lcl_95, ucl_95 = np.percentile(data, [2.5, 97.5])
        
        param_name = param.replace('_', '').replace('.pickle', '')
        
        # 数値を有効数字2桁にフォーマット
        if param_name not in ['z_1', 'z_2']:
            median = round(median, 2)
            lcl_68 = round(lcl_68, 2)
            ucl_68 = round(ucl_68, 2)
            lcl_95 = round(lcl_95, 2)
            ucl_95 = round(ucl_95, 2)
        
        results.append({
            'Parameter': param_name,
            'Units': units[param_name],
            'Median': median,
            '68% LCL': lcl_68,
            '68% UCL': ucl_68,
            # '95% LCL': lcl_95,
            # '95% UCL': ucl_95
        })

# 結果をDataFrameに変換
df_results = pd.DataFrame(results)

display(df_results)

# TSM0とTSM1に対応する行の抽出
tsm0_row = df_results[df_results['Parameter'] == 'TSM0']
tsm1_row = df_results[df_results['Parameter'] == 'TSM1']

# CSVに出力するための新しい行を作成
new_row = {
    'ClusterName': cluster_name,
    'MCMAC_TSC0': tsm0_row['Median'].values[0],
    'MCMAC_TSC0.lower': tsm0_row['68% LCL'].values[0],
    'MCMAC_TSC0.upper': tsm0_row['68% UCL'].values[0],
    'MCMAC_TSC1': tsm1_row['Median'].values[0],
    'MCMAC_TSC1.lower': tsm1_row['68% LCL'].values[0],
    'MCMAC_TSC1.upper': tsm1_row['68% UCL'].values[0],
    'Simulation_TSC': TSC_simulation,
}

# 新しい行をDataFrameに変換
new_row_df = pd.DataFrame([new_row])

# CSVファイルに出力
new_row_df.to_csv(result_file, index=False)

,Parameter,Units,Median,68% LCL,68% UCL
0,m1,10^14 M☉,4.54,3.77,5.34
1,m2,10^14 M☉,2.79,2.09,3.47
2,z1,,0.30,0.30,0.30
3,z2,,0.30,0.30,0.30
4,dproj,Mpc,0.44,0.42,0.46
5,vradobs,km s^{-1},92.68,24.88,206.90
6,alpha,degree,7.01,2.88,11.07
7,v3dobs,km s^{-1},952.79,261.08,1864.66
8,d3d,Mpc,0.44,0.42,0.46
9,v3dcol,km s^{-1},1423.65,1105.24,2137.85
